In [6]:
import pandas as pd
from pathlib import Path
import torch

In [7]:
FILE = Path("../../").absolute().resolve()
DATADIR = FILE / "data"
path = DATADIR / 'brazilian_houses' / str(1)

X_train_path = path / "X_train.parquet"
y_train_path = path / "y_train.parquet"
# X_test_path = path / "X_test.parquet"
# y_test_path = path / "y_test.parquet"

In [8]:
dataset = "bike_share_demand"
fold = str(1)

X = pd.read_parquet(X_train_path)
y = pd.read_parquet(y_train_path).iloc[:, 0]

In [9]:
model = torch.load("meta_model.pth", weights_only=False)

In [11]:
print(model)

{'model_state_dict': OrderedDict([('encoder.0.weight', tensor([[-4.1670e-02,  7.1547e-03, -1.3277e-01,  ...,  2.4904e-07,
         -8.5422e-05, -3.0629e-02],
        [ 1.1206e-01, -6.3058e-02, -1.4844e-01,  ...,  6.8688e-08,
          1.2046e-01,  1.2475e-01],
        [ 1.7652e-01, -4.7932e-02, -1.0690e-01,  ..., -2.5593e-07,
          4.1545e-03,  2.7546e-02],
        ...,
        [ 5.9922e-02,  8.3964e-02,  5.3604e-02,  ...,  2.6614e-07,
         -1.2531e-04,  2.7919e-03],
        [-7.3739e-02, -1.6441e-01, -1.5593e-02,  ...,  3.6142e-08,
         -2.3487e-04, -1.8271e-02],
        [ 4.1473e-02,  1.7159e-01, -6.0685e-02,  ..., -2.7386e-07,
          2.0635e-02,  1.0353e-01]])), ('encoder.0.bias', tensor([-8.1845e-07,  1.9188e-07, -8.0561e-07, -5.3351e-07,  2.3363e-07,
         1.9830e-07,  1.5638e-07, -1.4242e-07,  4.7628e-07,  1.6814e-07,
        -1.0068e-07,  5.2144e-08,  2.3770e-07, -4.1597e-08, -2.3000e-07,
        -1.3255e-07,  2.7373e-08, -4.8816e-08,  7.0127e-08, -3.2467e-07,


In [10]:
from meta_features import extract_meta_features


meta_features = extract_meta_features(X, y)
prediction = model.predict(meta_features)
print(prediction)

=========== Extracting meta-features ===========
 X Type: <class 'pandas.core.frame.DataFrame'> Shape: (4329, 11)
 y Type: <class 'pandas.core.series.Series'> Shape: (4329,)
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 30, number of used features: 0
[LightGBM] [Info] Start training from score 8.306491
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning

c:\ProgramData\miniconda3\envs\amlexam\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\ProgramData\miniconda3\envs\amlexam\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


AttributeError: 'dict' object has no attribute 'predict'

In [13]:
from meta_trainer import load_ranking_meta_model, predict_algorithm_rankings
models, checkpoint = load_ranking_meta_model("meta_model.pth")

meta_df = extract_meta_features(X,y)

predictions = predict_algorithm_rankings(models, checkpoint,meta_features)

Loaded XGBoost ranking model:
  Model type: XGBoost
  Number of algorithms: 10
  Number of features: 22
  Test Spearman correlation: 1.0000
  Test Top-1 accuracy: 1.0000
  Number of datasets: 19
=========== Extracting meta-features ===========
 X Type: <class 'pandas.core.frame.DataFrame'> Shape: (4329, 11)
 y Type: <class 'pandas.core.series.Series'> Shape: (4329,)
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 30, number of used features: 0
[LightGBM] [Info] Start training from score 8.257214
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped tra

c:\ProgramData\miniconda3\envs\amlexam\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\ProgramData\miniconda3\envs\amlexam\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


TypeError: unhashable type: 'list'

In [1]:
# Parse meta_preds.csv to analyze top 4 algorithm rankings
import pandas as pd

# Load the predictions CSV
preds_df = pd.read_csv("meta_preds.csv", index_col=0)

# Get algorithm names by finding unique prefixes of ranking columns
actual_cols = [col for col in preds_df.columns if col.endswith('_actual_rank')]
predicted_cols = [col for col in preds_df.columns if col.endswith('_predicted_rank')]

# Extract algorithm names
algo_names = [col.replace('_actual_rank', '') for col in actual_cols]

print(f"Found {len(algo_names)} algorithms: {algo_names}")
print(f"Analyzing {len(preds_df)} records\n")

# Function to get top K algorithms from rankings
def get_top_k_algorithms(row, rank_suffix, k=4):
    """Get top K algorithms based on their rankings (1=best, 2=second best, etc.)"""
    algo_ranks = {}
    for algo in algo_names:
        col_name = f"{algo}{rank_suffix}"
        if col_name in row:
            algo_ranks[algo] = row[col_name]
    
    # Sort by rank (ascending: 1, 2, 3, 4...)
    sorted_algos = sorted(algo_ranks.items(), key=lambda x: x[1])
    return [algo for algo, rank in sorted_algos[:k]]

# Analyze each record
results = []
for idx, row in preds_df.iterrows():
    top4_actual = get_top_k_algorithms(row, '_actual_rank', k=4)
    top4_predicted = get_top_k_algorithms(row, '_predicted_rank', k=4)
    
    # Calculate overlap (how many algorithms appear in both top 4 lists)
    overlap = len(set(top4_actual).intersection(set(top4_predicted)))
    
    # Check if top 1 matches
    top1_match = top4_actual[0] == top4_predicted[0] if top4_actual and top4_predicted else False
    
    results.append({
        'record': idx,
        'top4_actual': top4_actual,
        'top4_predicted': top4_predicted,
        'top1_actual': top4_actual[0] if top4_actual else None,
        'top1_predicted': top4_predicted[0] if top4_predicted else None,
        'top1_match': top1_match,
        'top4_overlap': overlap,
        'top4_overlap_pct': (overlap / 4) * 100
    })

# Convert to DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Display first few records
print("Sample Results:")
for i in range(min(5, len(results_df))):
    row = results_df.iloc[i]
    print(f"\nRecord {row['record']}:")
    print(f"  Top 4 Actual:    {row['top4_actual']}")
    print(f"  Top 4 Predicted: {row['top4_predicted']}")
    print(f"  Top 1 Match: {'✓' if row['top1_match'] else '✗'}")
    print(f"  Top 4 Overlap: {row['top4_overlap']}/4 ({row['top4_overlap_pct']:.0f}%)")

# Summary statistics
print(f"\n{'='*50}")
print("SUMMARY STATISTICS")
print(f"{'='*50}")
print(f"Top 1 Accuracy: {results_df['top1_match'].mean():.3f} ({results_df['top1_match'].mean()*100:.1f}%)")
print(f"Average Top 4 Overlap: {results_df['top4_overlap'].mean():.2f}/4 ({results_df['top4_overlap_pct'].mean():.1f}%)")
print(f"Perfect Top 4 Match: {(results_df['top4_overlap'] == 4).sum()}/{len(results_df)} records")
print(f"At least 3/4 match: {(results_df['top4_overlap'] >= 3).sum()}/{len(results_df)} records")

# Save detailed results
results_df.to_csv("top4_analysis.csv", index=False)
print(f"\nDetailed analysis saved to: top4_analysis.csv")

Found 10 algorithms: ['BayesianRidge', 'LinearRegression', 'SVR', 'RandomForestRegressor', 'GradientBoostingRegressor', 'LGBMRegressor', 'HistGradientBoostingRegressor', 'XGBRegressor', 'DecisionTreeRegressor', 'MLPRegressor']
Analyzing 200 records

Sample Results:

Record 521:
  Top 4 Actual:    ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTreeRegressor']
  Top 4 Predicted: ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTreeRegressor']
  Top 1 Match: ✓
  Top 4 Overlap: 4/4 (100%)

Record 737:
  Top 4 Actual:    ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTreeRegressor']
  Top 4 Predicted: ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTreeRegressor']
  Top 1 Match: ✓
  Top 4 Overlap: 4/4 (100%)

Record 740:
  Top 4 Actual:    ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTreeRegressor']
  Top 4 Predicted: ['LGBMRegressor', 'XGBRegressor', 'RandomForestRegressor', 'DecisionTre

In [2]:
# Show only records where top 4 exact order doesn't match
import pandas as pd

# Load the predictions CSV
preds_df = pd.read_csv("meta_preds.csv", index_col=0)

# Get algorithm names by finding unique prefixes of ranking columns
actual_cols = [col for col in preds_df.columns if col.endswith('_actual_rank')]
predicted_cols = [col for col in preds_df.columns if col.endswith('_predicted_rank')]

# Extract algorithm names
algo_names = [col.replace('_actual_rank', '') for col in actual_cols]

print(f"Found {len(algo_names)} algorithms: {algo_names}")
print(f"Analyzing {len(preds_df)} records\n")

# Function to get top K algorithms in exact rank order
def get_top_k_algorithms_ordered(row, rank_suffix, k=4):
    """Get top K algorithms in exact rank order (rank 1, rank 2, etc.)"""
    algo_ranks = {}
    for algo in algo_names:
        col_name = f"{algo}{rank_suffix}"
        if col_name in row:
            algo_ranks[algo] = row[col_name]
    
    # Sort by rank (ascending: 1, 2, 3, 4...)
    sorted_algos = sorted(algo_ranks.items(), key=lambda x: x[1])
    return [algo for algo, rank in sorted_algos[:k]]

# Find records with mismatched top 4 exact order
mismatched_records = []
total_records = 0
exact_matches = 0

for idx, row in preds_df.iterrows():
    total_records += 1
    top4_actual = get_top_k_algorithms_ordered(row, '_actual_rank', k=4)
    top4_predicted = get_top_k_algorithms_ordered(row, '_predicted_rank', k=4)
    
    # Check if exact order matches
    exact_match = top4_actual == top4_predicted
    
    if exact_match:
        exact_matches += 1
    else:
        # Store mismatched record
        mismatched_records.append({
            'record': idx,
            'top4_actual': top4_actual,
            'top4_predicted': top4_predicted
        })

print(f"EXACT TOP 4 ORDER ANALYSIS")
print(f"{'='*50}")
print(f"Total records: {total_records}")
print(f"Exact top 4 matches: {exact_matches}")
print(f"Mismatched records: {len(mismatched_records)}")
print(f"Exact match rate: {(exact_matches/total_records)*100:.1f}%")

print(f"\n{'='*60}")
print(f"RECORDS WITH MISMATCHED TOP 4 ORDER")
print(f"{'='*60}")

if len(mismatched_records) == 0:
    print("🎉 All records have perfect top 4 order matches!")
else:
    for i, record in enumerate(mismatched_records):
        print(f"\nRecord {record['record']} (Mismatch #{i+1}):")
        print(f"  Actual Top 4:    {' → '.join([f'{j+1}. {algo}' for j, algo in enumerate(record['top4_actual'])])}")
        print(f"  Predicted Top 4: {' → '.join([f'{j+1}. {algo}' for j, algo in enumerate(record['top4_predicted'])])}")
        
        # Show position-wise differences
        differences = []
        for pos in range(4):
            if pos < len(record['top4_actual']) and pos < len(record['top4_predicted']):
                if record['top4_actual'][pos] != record['top4_predicted'][pos]:
                    differences.append(f"Pos {pos+1}: {record['top4_actual'][pos]} ≠ {record['top4_predicted'][pos]}")
        
        if differences:
            print(f"  Differences:     {' | '.join(differences)}")

# Save mismatched records for further analysis
if mismatched_records:
    mismatch_df = pd.DataFrame(mismatched_records)
    mismatch_df.to_csv("mismatched_top4_records.csv", index=False)
    print(f"\nMismatched records saved to: mismatched_top4_records.csv")

Found 10 algorithms: ['BayesianRidge', 'LinearRegression', 'SVR', 'RandomForestRegressor', 'GradientBoostingRegressor', 'LGBMRegressor', 'HistGradientBoostingRegressor', 'XGBRegressor', 'DecisionTreeRegressor', 'MLPRegressor']
Analyzing 200 records

EXACT TOP 4 ORDER ANALYSIS
Total records: 200
Exact top 4 matches: 184
Mismatched records: 16
Exact match rate: 92.0%

RECORDS WITH MISMATCHED TOP 4 ORDER

Record 76 (Mismatch #1):
  Actual Top 4:    1. RandomForestRegressor → 2. BayesianRidge → 3. GradientBoostingRegressor → 4. XGBRegressor
  Predicted Top 4: 1. LinearRegression → 2. RandomForestRegressor → 3. BayesianRidge → 4. LGBMRegressor
  Differences:     Pos 1: RandomForestRegressor ≠ LinearRegression | Pos 2: BayesianRidge ≠ RandomForestRegressor | Pos 3: GradientBoostingRegressor ≠ BayesianRidge | Pos 4: XGBRegressor ≠ LGBMRegressor

Record 59 (Mismatch #2):
  Actual Top 4:    1. LGBMRegressor → 2. BayesianRidge → 3. LinearRegression → 4. GradientBoostingRegressor
  Predicted Top 

In [3]:
# Meta-Model Diversity Evaluation using existing predictions
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

def evaluate_existing_predictions():
    """Evaluate meta-model diversity using existing meta_preds.csv"""
    
    # Load existing predictions
    preds_df = pd.read_csv("meta_preds.csv", index_col=0)
    
    print("="*60)
    print("META-MODEL DIVERSITY EVALUATION")
    print("="*60)
    print(f"Analyzing {len(preds_df)} existing predictions\n")
    
    # Extract recommended algorithms
    if 'recommended_algorithm' in preds_df.columns:
        recommendations = preds_df['recommended_algorithm'].tolist()
    else:
        # If no recommended_algorithm column, extract from rank 1 predictions
        algo_names = [col.replace('_predicted_rank', '') for col in preds_df.columns if col.endswith('_predicted_rank')]
        recommendations = []
        
        for idx, row in preds_df.iterrows():
            # Find algorithm with rank 1 (best prediction)
            best_algo = None
            for algo in algo_names:
                if f"{algo}_predicted_rank" in row and row[f"{algo}_predicted_rank"] == 1:
                    best_algo = algo
                    break
            recommendations.append(best_algo)
    
    # Count algorithm frequency
    algo_frequency = Counter(recommendations)
    total_predictions = len(recommendations)
    unique_algorithms = len(algo_frequency)
    
    print("ALGORITHM RECOMMENDATION FREQUENCY:")
    print("-" * 40)
    for algo, count in sorted(algo_frequency.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_predictions) * 100
        bar = "█" * int(percentage / 2)  # Simple bar chart
        print(f"{algo:20} | {count:3d} ({percentage:5.1f}%) {bar}")
    
    print(f"\n{'='*50}")
    print("DIVERSITY METRICS")
    print(f"{'='*50}")
    
    # Calculate diversity metrics
    diversity_ratio = unique_algorithms / len(set([algo for algo in algo_frequency.keys() if algo is not None]))
    most_frequent = algo_frequency.most_common(1)[0] if algo_frequency else (None, 0)
    concentration = most_frequent[1] / total_predictions if most_frequent[1] > 0 else 0
    
    print(f"Total predictions analyzed: {total_predictions}")
    print(f"Unique algorithms recommended: {unique_algorithms}")
    print(f"Most frequent algorithm: {most_frequent[0]} ({concentration*100:.1f}%)")
    
    # Diversity assessment
    print(f"\nDIVERSITY ASSESSMENT:")
    if concentration > 0.7:
        print(f"❌ POOR DIVERSITY: {most_frequent[0]} dominates {concentration*100:.1f}% of predictions")
        print("   → Meta-model may be over-fitted to one algorithm")
        assessment = "POOR"
    elif concentration > 0.5:
        print(f"⚠️  MODERATE DIVERSITY: {most_frequent[0]} used in {concentration*100:.1f}% of cases")
        print("   → Some algorithm variety, but room for improvement")
        assessment = "MODERATE"
    elif unique_algorithms < 3:
        print(f"⚠️  LIMITED VARIETY: Only {unique_algorithms} algorithms recommended")
        print("   → Meta-model not utilizing full algorithm portfolio")
        assessment = "LIMITED"
    else:
        print(f"✅ GOOD DIVERSITY: {unique_algorithms} algorithms used, max concentration {concentration*100:.1f}%")
        print("   → Meta-model shows dataset-specific behavior")
        assessment = "GOOD"
    
    # Entropy calculation (information theory measure of diversity)
    probabilities = [count/total_predictions for count in algo_frequency.values()]
    entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
    max_entropy = np.log2(len(algo_frequency))
    normalized_entropy = entropy / max_entropy if max_entropy > 0 else 0
    
    print(f"\nINFORMATION THEORY METRICS:")
    print(f"Shannon Entropy: {entropy:.3f}")
    print(f"Normalized Entropy: {normalized_entropy:.3f} (1.0 = perfectly diverse)")
    
    if normalized_entropy > 0.8:
        print("✅ High entropy - Very diverse predictions")
    elif normalized_entropy > 0.6:
        print("⚠️  Moderate entropy - Some diversity")
    else:
        print("❌ Low entropy - Poor diversity")
    
    return {
        'assessment': assessment,
        'concentration': concentration,
        'entropy': normalized_entropy,
        'unique_algorithms': unique_algorithms,
        'most_frequent': most_frequent[0],
        'frequency_distribution': dict(algo_frequency)
    }

def analyze_algorithm_patterns():
    """Analyze which algorithms are predicted for what types of datasets"""
    
    preds_df = pd.read_csv("meta_preds.csv", index_col=0)
    
    print("\n" + "="*60)
    print("ALGORITHM PATTERN ANALYSIS")
    print("="*60)
    
    # If meta-features are available in the predictions file
    meta_feature_cols = [col for col in preds_df.columns if not col.endswith('_rank') and 
                        col != 'recommended_algorithm' and col != 'actual_best_algorithm']
    
    if len(meta_feature_cols) > 0:
        print(f"Found {len(meta_feature_cols)} meta-features for pattern analysis")
        
        # Group by recommended algorithm
        if 'recommended_algorithm' in preds_df.columns:
            algo_groups = preds_df.groupby('recommended_algorithm')
            
            print(f"\nPATTERN ANALYSIS BY ALGORITHM:")
            print("-" * 50)
            
            for algo, group in algo_groups:
                if len(group) < 2:  # Skip algorithms with only 1 prediction
                    continue
                    
                print(f"\n{algo} ({len(group)} datasets):")
                
                # Analyze key meta-features
                key_features = ['n_samples', 'n_features', 'feature_ratio', 'target_mean', 'target_std']
                available_features = [f for f in key_features if f in meta_feature_cols]
                
                for feature in available_features:
                    if feature in group.columns:
                        mean_val = group[feature].mean()
                        std_val = group[feature].std()
                        print(f"  {feature}: {mean_val:.3f} ± {std_val:.3f}")
    else:
        print("No meta-features found in predictions file for pattern analysis")
    
    # Analyze actual vs predicted performance correlation
    if 'actual_best_algorithm' in preds_df.columns and 'recommended_algorithm' in preds_df.columns:
        correct_predictions = (preds_df['actual_best_algorithm'] == preds_df['recommended_algorithm']).sum()
        total_predictions = len(preds_df)
        accuracy = correct_predictions / total_predictions
        
        print(f"\nPREDICTION ACCURACY:")
        print(f"Correct predictions: {correct_predictions}/{total_predictions} ({accuracy*100:.1f}%)")
        
        # Analyze which algorithms are correctly predicted most often
        correct_mask = preds_df['actual_best_algorithm'] == preds_df['recommended_algorithm']
        correct_algos = preds_df[correct_mask]['recommended_algorithm'].value_counts()
        
        print(f"\nMOST ACCURATELY PREDICTED ALGORITHMS:")
        for algo, count in correct_algos.head().items():
            total_algo_predictions = (preds_df['recommended_algorithm'] == algo).sum()
            algo_accuracy = count / total_algo_predictions if total_algo_predictions > 0 else 0
            print(f"  {algo}: {count}/{total_algo_predictions} ({algo_accuracy*100:.1f}%)")

def quick_diversity_check():
    """Quick one-line diversity check"""
    
    preds_df = pd.read_csv("meta_preds.csv", index_col=0)
    
    if 'recommended_algorithm' in preds_df.columns:
        recommendations = preds_df['recommended_algorithm']
        algo_counts = recommendations.value_counts()
        
        print("QUICK DIVERSITY CHECK:")
        print(f"🔢 Total predictions: {len(recommendations)}")
        print(f"🎯 Unique algorithms: {len(algo_counts)}")
        print(f"👑 Most frequent: {algo_counts.index[0]} ({algo_counts.iloc[0]/len(recommendations)*100:.1f}%)")
        
        if algo_counts.iloc[0]/len(recommendations) > 0.7:
            print("❌ DIVERSITY ISSUE: One algorithm dominates!")
        else:
            print("✅ ACCEPTABLE DIVERSITY")

# Run the evaluation
print("EVALUATING EXISTING META-MODEL PREDICTIONS")
print("=" * 70)

# Quick check first
quick_diversity_check()

# Detailed evaluation
results = evaluate_existing_predictions()

# Pattern analysis
analyze_algorithm_patterns()

print(f"\n{'='*50}")
print("RECOMMENDATIONS")
print(f"{'='*50}")

if results['assessment'] == 'POOR':
    print("🔧 ACTIONS NEEDED:")
    print("  1. Check training data balance - ensure diverse algorithm performance")
    print("  2. Adjust loss function to penalize over-concentration")
    print("  3. Use dataset stratification during meta-model training")
    print("  4. Consider ensemble of specialized meta-models")

elif results['assessment'] == 'MODERATE':
    print("🔧 IMPROVEMENTS SUGGESTED:")
    print("  1. Add more diverse training datasets")
    print("  2. Fine-tune meta-feature extraction")
    print("  3. Consider regularization to prevent algorithm bias")

else:
    print("✅ GOOD PERFORMANCE:")
    print("  Meta-model shows good algorithm diversity")
    print("  Continue monitoring on new datasets")

print(f"\nEvaluation complete! Results based on {len(pd.read_csv('meta_preds.csv'))} predictions.")

EVALUATING EXISTING META-MODEL PREDICTIONS
QUICK DIVERSITY CHECK:
🔢 Total predictions: 200
🎯 Unique algorithms: 6
👑 Most frequent: LGBMRegressor (96.0%)
❌ DIVERSITY ISSUE: One algorithm dominates!
META-MODEL DIVERSITY EVALUATION
Analyzing 200 existing predictions

ALGORITHM RECOMMENDATION FREQUENCY:
----------------------------------------
LGBMRegressor        | 192 ( 96.0%) ████████████████████████████████████████████████
LinearRegression     |   3 (  1.5%) 
RandomForestRegressor |   2 (  1.0%) 
BayesianRidge        |   1 (  0.5%) 
GradientBoostingRegressor |   1 (  0.5%) 
XGBRegressor         |   1 (  0.5%) 

DIVERSITY METRICS
Total predictions analyzed: 200
Unique algorithms recommended: 6
Most frequent algorithm: LGBMRegressor (96.0%)

DIVERSITY ASSESSMENT:
❌ POOR DIVERSITY: LGBMRegressor dominates 96.0% of predictions
   → Meta-model may be over-fitted to one algorithm

INFORMATION THEORY METRICS:
Shannon Entropy: 0.329
Normalized Entropy: 0.127 (1.0 = perfectly diverse)
❌ Low ent

In [4]:
# Analyze Algorithm Diversity in Top 4 Ranks from meta_Y.csv
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations
import matplotlib.pyplot as plt

def analyze_top4_diversity():
    """Analyze diversity of top 4 algorithm rankings in meta_Y.csv"""
    
    # Load the meta_Y.csv file
    try:
        meta_y_df = pd.read_csv("meta_Y.csv", index_col=0)
        print(f"Loaded meta_Y.csv with {len(meta_y_df)} records")
    except FileNotFoundError:
        print("❌ meta_Y.csv not found!")
        return
    
    print("="*60)
    print("TOP 4 ALGORITHM DIVERSITY ANALYSIS")
    print("="*60)
    
    # Get algorithm names by finding columns ending with '_rank'
    rank_columns = [col for col in meta_y_df.columns if col.endswith('_rank')]
    algo_names = [col.replace('_rank', '') for col in rank_columns]
    
    print(f"Found {len(algo_names)} algorithms: {algo_names}")
    print(f"Analyzing {len(meta_y_df)} records\n")
    
    # Function to get top K algorithms for each record
    def get_top_k_algorithms(row, k=4):
        """Get top K algorithms based on their rankings (1=best, 2=second best, etc.)"""
        algo_ranks = {}
        for algo in algo_names:
            col_name = f"{algo}_rank"
            if col_name in row:
                algo_ranks[algo] = row[col_name]
        
        # Sort by rank (ascending: 1, 2, 3, 4...)
        sorted_algos = sorted(algo_ranks.items(), key=lambda x: x[1])
        return [algo for algo, rank in sorted_algos[:k]]
    
    # Extract top 4 for each record
    all_top4_sets = []
    rank_position_algorithms = {1: [], 2: [], 3: [], 4: []}  # Track which algo appears at each position
    
    for idx, row in meta_y_df.iterrows():
        top4 = get_top_k_algorithms(row, k=4)
        all_top4_sets.append(tuple(top4))  # Convert to tuple for hashing
        
        # Track position-specific algorithms
        for pos, algo in enumerate(top4[:4], 1):
            rank_position_algorithms[pos].append(algo)
    
    # 1. OVERALL TOP 4 SET DIVERSITY
    print("1. OVERALL TOP 4 SET DIVERSITY:")
    print("-" * 40)
    
    unique_top4_sets = set(all_top4_sets)
    set_frequency = Counter(all_top4_sets)
    
    print(f"Total records: {len(all_top4_sets)}")
    print(f"Unique top 4 combinations: {len(unique_top4_sets)}")
    print(f"Diversity ratio: {len(unique_top4_sets)/len(all_top4_sets):.3f}")
    
    # Show most common top 4 combinations
    print(f"\nMost frequent top 4 combinations:")
    for i, (top4_set, count) in enumerate(set_frequency.most_common(10)):
        percentage = (count / len(all_top4_sets)) * 100
        top4_str = " → ".join([f"{j+1}.{algo}" for j, algo in enumerate(top4_set)])
        print(f"  {i+1:2d}. {top4_str}")
        print(f"      Frequency: {count}/{len(all_top4_sets)} ({percentage:.1f}%)")
        if i < 5:  # Show details for top 5
            print()
    
    # 2. POSITION-SPECIFIC DIVERSITY
    print(f"\n2. POSITION-SPECIFIC DIVERSITY:")
    print("-" * 40)
    
    for position in [1, 2, 3, 4]:
        position_algos = rank_position_algorithms[position]
        algo_frequency = Counter(position_algos)
        unique_algos_at_position = len(algo_frequency)
        
        print(f"\nRank {position} (Best → 4th):")
        print(f"  Unique algorithms appearing: {unique_algos_at_position}/{len(algo_names)}")
        
        # Show top algorithms for this position
        for algo, count in algo_frequency.most_common(5):
            percentage = (count / len(position_algos)) * 100
            bar = "█" * min(int(percentage / 3), 20)  # Visual bar
            print(f"    {algo:20} | {count:3d} ({percentage:5.1f}%) {bar}")
        
        # Diversity assessment for this position
        most_frequent_pct = (algo_frequency.most_common(1)[0][1] / len(position_algos)) * 100
        if most_frequent_pct > 70:
            print(f"    ❌ POOR diversity: {algo_frequency.most_common(1)[0][0]} dominates {most_frequent_pct:.1f}%")
        elif most_frequent_pct > 50:
            print(f"    ⚠️  MODERATE diversity: {algo_frequency.most_common(1)[0][0]} appears {most_frequent_pct:.1f}%")
        else:
            print(f"    ✅ GOOD diversity: Most frequent is {most_frequent_pct:.1f}%")
    
    # 3. ALGORITHM INCLUSION FREQUENCY
    print(f"\n3. ALGORITHM INCLUSION IN TOP 4:")
    print("-" * 40)
    
    # Count how often each algorithm appears in ANY of the top 4 positions
    algo_inclusion_count = Counter()
    for top4_set in all_top4_sets:
        for algo in top4_set:
            algo_inclusion_count[algo] += 1
    
    print(f"How often each algorithm appears in top 4 (any position):")
    for algo, count in algo_inclusion_count.most_common():
        percentage = (count / len(all_top4_sets)) * 100
        bar = "█" * min(int(percentage / 2), 30)
        print(f"  {algo:20} | {count:3d}/{len(all_top4_sets)} ({percentage:5.1f}%) {bar}")
    
    # 4. PAIRWISE ALGORITHM CO-OCCURRENCE
    print(f"\n4. ALGORITHM CO-OCCURRENCE IN TOP 4:")
    print("-" * 40)
    
    # Find which algorithms frequently appear together in top 4
    pair_counts = Counter()
    for top4_set in all_top4_sets:
        # Get all pairs of algorithms in this top 4
        for algo1, algo2 in combinations(top4_set, 2):
            pair = tuple(sorted([algo1, algo2]))  # Sort to ensure consistent ordering
            pair_counts[pair] += 1
    
    print(f"Most frequent algorithm pairs in top 4:")
    for i, (pair, count) in enumerate(pair_counts.most_common(10)):
        percentage = (count / len(all_top4_sets)) * 100
        print(f"  {i+1:2d}. {pair[0]} + {pair[1]}: {count} times ({percentage:.1f}%)")
    
    # 5. DIVERSITY ASSESSMENT
    print(f"\n{'='*50}")
    print("DIVERSITY ASSESSMENT")
    print(f"{'='*50}")
    
    # Overall assessment
    overall_diversity = len(unique_top4_sets) / len(all_top4_sets)
    most_common_set_pct = (set_frequency.most_common(1)[0][1] / len(all_top4_sets)) * 100
    
    print(f"Overall Top 4 Diversity: {overall_diversity:.3f}")
    print(f"Most common combination appears: {most_common_set_pct:.1f}% of the time")
    
    if overall_diversity > 0.8:
        print("✅ EXCELLENT: Very high diversity in top 4 combinations")
    elif overall_diversity > 0.5:
        print("✅ GOOD: Good diversity in top 4 combinations")
    elif overall_diversity > 0.3:
        print("⚠️  MODERATE: Some diversity, but room for improvement")
    else:
        print("❌ POOR: Low diversity - similar algorithms always in top 4")
    
    # Position-specific assessment
    print(f"\nPosition-specific diversity:")
    for pos in [1, 2, 3, 4]:
        pos_algos = rank_position_algorithms[pos]
        pos_diversity = len(set(pos_algos)) / len(algo_names)
        most_frequent_at_pos = Counter(pos_algos).most_common(1)[0]
        dominance_pct = (most_frequent_at_pos[1] / len(pos_algos)) * 100
        
        print(f"  Rank {pos}: {len(set(pos_algos))}/{len(algo_names)} unique algos, {most_frequent_at_pos[0]} leads ({dominance_pct:.1f}%)")
    
    # 6. RECOMMENDATIONS
    print(f"\n{'='*50}")
    print("RECOMMENDATIONS")
    print(f"{'='*50}")
    
    if overall_diversity < 0.3:
        print("🔧 URGENT ACTIONS NEEDED:")
        print("  1. Training data lacks diversity - same algorithms always win")
        print("  2. Add datasets that favor different algorithm combinations")
        print("  3. Check if meta-features capture enough dataset variety")
        print("  4. Consider synthetic dataset generation for rare cases")
    elif overall_diversity < 0.5:
        print("🔧 IMPROVEMENTS SUGGESTED:")
        print("  1. Increase dataset variety in training")
        print("  2. Ensure balanced algorithm performance across datasets")
        print("  3. Check for data collection bias")
    else:
        print("✅ GOOD DIVERSITY:")
        print("  Your meta-learning data shows good algorithm variety")
        print("  Continue monitoring for edge cases")
    
    # Check for specific algorithm dominance issues
    dominant_algos = []
    for pos in [1, 2, 3, 4]:
        pos_counter = Counter(rank_position_algorithms[pos])
        most_freq = pos_counter.most_common(1)[0]
        if (most_freq[1] / len(rank_position_algorithms[pos])) > 0.6:
            dominant_algos.append((pos, most_freq[0], most_freq[1] / len(rank_position_algorithms[pos]) * 100))
    
    if dominant_algos:
        print(f"\n⚠️  DOMINANCE ISSUES DETECTED:")
        for pos, algo, pct in dominant_algos:
            print(f"  Rank {pos}: {algo} appears {pct:.1f}% of the time")
        print("  → Need more diverse datasets to break algorithm dominance")
    
    return {
        'overall_diversity': overall_diversity,
        'unique_combinations': len(unique_top4_sets),
        'most_common_percentage': most_common_set_pct,
        'algorithm_inclusion': dict(algo_inclusion_count),
        'position_diversity': {pos: len(set(rank_position_algorithms[pos])) for pos in [1,2,3,4]}
    }

# Run the analysis
print("ANALYZING ALGORITHM DIVERSITY IN META_Y.CSV")
print("=" * 70)

results = analyze_top4_diversity()

if results:
    print(f"\n🔍 QUICK SUMMARY:")
    print(f"   • Overall diversity: {results['overall_diversity']:.3f}")
    print(f"   • Unique top-4 combinations: {results['unique_combinations']}")
    print(f"   • Most common combo appears: {results['most_common_percentage']:.1f}% of time")
    print(f"   • Algorithm inclusion range: {min(results['algorithm_inclusion'].values())} - {max(results['algorithm_inclusion'].values())} times")

ANALYZING ALGORITHM DIVERSITY IN META_Y.CSV
Loaded meta_Y.csv with 1000 records
TOP 4 ALGORITHM DIVERSITY ANALYSIS
Found 10 algorithms: ['BayesianRidge', 'LinearRegression', 'SVR', 'RandomForestRegressor', 'GradientBoostingRegressor', 'LGBMRegressor', 'HistGradientBoostingRegressor', 'XGBRegressor', 'DecisionTreeRegressor', 'MLPRegressor']
Analyzing 1000 records

1. OVERALL TOP 4 SET DIVERSITY:
----------------------------------------
Total records: 1000
Unique top 4 combinations: 62
Diversity ratio: 0.062

Most frequent top 4 combinations:
   1. 1.LGBMRegressor → 2.XGBRegressor → 3.RandomForestRegressor → 4.DecisionTreeRegressor
      Frequency: 932/1000 (93.2%)

   2. 1.RandomForestRegressor → 2.GradientBoostingRegressor → 3.DecisionTreeRegressor → 4.HistGradientBoostingRegressor
      Frequency: 3/1000 (0.3%)

   3. 1.BayesianRidge → 2.LinearRegression → 3.SVR → 4.RandomForestRegressor
      Frequency: 2/1000 (0.2%)

   4. 1.MLPRegressor → 2.RandomForestRegressor → 3.LGBMRegressor →